# CS 425 fall 2021 Assignment 5


## Read mapping

In this assignment you will implement a read mapper based on suffix arrays and test it on a small dataset from the bacterium E. coli.

The sequencing reads are in [FASTQ format](https://en.wikipedia.org/wiki/FASTQ_format).  Each FASTQ record is composed of four lines; for this work we will use the first line which contains read identifiers, and the third line which contains the sequence of the read.
The reads should be aligned against the reference genome of the bacterium [E. coli](https://www.ncbi.nlm.nih.gov/nuccore/NC_000913).

Note that each read can originate from the positive or negative strand.  So take that into account in the mapping process.

For the output, create a tab delimited file, with a line for each read that you were able to match.  The columns for the file should be as follows:

* Read identifier (from the FASTQ file)
* The strand to which the read has been matched, represented by the symbols + or -.
* The position of the first matching nucleotide in 0-based indexing.

In you implementation use the read-mapping strategy we discussed in class that accounts for the possibility for errors by splitting a read into $k+1$ segments when allowing for up to $k$ errors.


In [2]:
!pip install pandas
import pandas as pd
import csv
from collections import defaultdict

Below is code for computing the suffix array that you can use in your code.  It has been tested to work for the *E. coli* genome in a very reasonable amount of time.

In [3]:
def read_fasta(file_name):
    with open(file_name,"r") as f:
    #read first line of description
        description = f.readline().rstrip()

    #read genome sequence
        genome = ""
        line = f.readline().rstrip()
        while line != '':
            genome += line
            line = f.readline().rstrip()

    return genome + '$'

def read_FASTQ(file_name):
  headers = []
  sequences = []
  seps = []
  qualities = []

  final = []

  # add each section of the FASTQ file into its own list
  with open(file_name, 'r') as ifh:
    for header, sequence, sep, quality in zip(ifh, ifh, ifh, ifh):
      headers.append(header.rstrip())
      sequences.append(sequence.rstrip())
      seps.append(sep.rstrip())
      qualities.append(quality.rstrip())

  # take all lists and combine them into a 2d list
  for i in range(len(headers)):
    # temp = [headers[i], sequences[i], seps[i], qualities[i]]
    temp = [headers[i], sequences[i], seps[i]]
    final.append(temp)

  return final

# part of creating suffix array
def bucket_sort(s, bucket, order):
    d = defaultdict(list)
    for i in bucket:
        key = s[i + order // 2:i + order]
        d[key].append(i)
    result = []
    for k, v in sorted(d.items()):
        if len(v) > 1:
            result += bucket_sort(s, v, 2 * order)
        else:
            result.append(v[0])
    return result

# Searches through suffix Text to see if pattern is part of Text
def binary_Search(sfxSorted, pattern, num_errors=0):
  start = 0
  end = len(sfxSorted) - 1
  length = len(pattern)


  while start <= end:
    middle = (start + end) // 2
    test = sfxSorted[middle]
    midpoint = test[:length]

    # Check number of errors
    if (sum(c1!=c2 for c1,c2 in zip(pattern,midpoint)) == num_errors and num_errors > 0):
      split_pattern = divide_string(pattern, num_errors+1)
      split_text = divide_string(midpoint, num_errors+1)
      
      for i in range(len(split_text)):
        if split_pattern[i] == split_text[i]:
          return binary_Search(sfxSorted, split_pattern[i], 0)

    # Else if midpoint is greater, ignore top/left half
    elif(midpoint  > pattern):
      end = middle - 1
    
    # Else if x is smaller, ignore bottom/right half
    elif( midpoint < pattern):
      start = middle + 1
    
    # Checks if pattern is present at mid
    else:
      return top_search(sfxSorted, pattern, middle)

  return -1

# Finds first occurence of pattern inside Suffix Text
def top_search(sfxSorted, pattern, middle):
  shorten = [x[:len(pattern)] for x in sfxSorted]
  shorten = shorten[:middle+1]

  end = len(shorten)-1
  firstOccur = end

  while pattern == shorten[end]:
    firstOccur = end
    end -= 1
  
  return firstOccur

# If mismatch found then it breaks string up into n pieces
def divide_string(string, n):
  chunk = int(len(string) / n) + (len(string) % n > 0)
  start = 0
  store = []
  for i in range(n):
    if i == n-1:
      store.append(string[start:])
    else:
      store.append(string[start:start+chunk])
    start += chunk

  return store

# Reverse Complement
def complement(genome):
  bases = {'A':'T','T':'A','C':'G','G':'C'}
  rev = []
  for gene in genome:
    rev += bases[gene]
    
  return ''.join(rev)[::-1]

# creates Array of suffix index
def suffix_array(s):
  return bucket_sort(s, range(len(s)), 1)

# Creates Array of suffix Text
def sorted_suffix(text, sfxArray):
  return [text[i:] for i in sfxArray]

def map_reads(text, patterns, output_file, num_errors=0, first=True) :
    # Add to the end of Genome
    text += '$'
    
    # create output file
    if first:
      df = pd.DataFrame(list())
      df.to_csv(output_file)
      with open(output_file, 'w', newline='') as file:
        col_names = ['Id', 'Pattern', 'Index(0 Based)']
        writer = csv.writer(file)
        writer.writerow(col_names)
        file.close()

    # Create suffix Array's
    sfxArray = suffix_array(text)
    print('finished suffix array')
    sfxSorted = sorted_suffix(text, sfxArray)
    print('finished suffix sorted')

    for pattern in patterns:
      position = binary_Search(sfxSorted, pattern[1], num_errors)
      
      if(position != -1):
        with open(output_file, 'a', newline='') as file:
          writer = csv.writer(file)
          if first:
            temp = [pattern[0], pattern[2], sfxArray[position]]
          else:
            temp = [pattern[0], '-', sfxArray[position]] 
            
          writer.writerow(temp)
          file.close()
    return 1


In [ ]:
if __name__== '__main__':
  genome = read_fasta("sequence.fasta") # TEXT
  r_genome = read_fasta("sequence.fasta")
  fastq = read_FASTQ("ecoli.fastq") # PATTERNS

  map_reads(genome[:3000], fastq, 'output.csv', 0, True)
  
  # Reverse Complement
  r_genome = complement(r_genome[:len(r_genome)-1])
  map_reads(r_genome[:3000], fastq, 'output.csv', 0, False)

### Data analysis

Run your code on the *E. coli* dataset, varying the number of allowed errors between 0 and 3.  Report how the number of reads that you are able to map increases when increasing the number of allowed errors.  How does that match what you know about error rates in Illumina sequencing?

# Data Collection
>For today's analysis I will be using two diffrent files. The first file I want to talk about it the fasta file. The file contains the reference genome of E. coli. The 2nd file is a FastQ file that is compossed of four lines, the first being the sequence identifier. The second line is the raw sequence letters which is followed by a '+' or '-' character. Lastly it has an encoded quality value for the raw sequence in line 2. The FastQ file contains 2500 patterns that I will need to see if they appear in the reference genome. The size of the reference genome is roughly about four million characters. With these two files I will create a Read Mapping Program on Python.

# Data Quality Check and Cleaning
> I've worked with Fasta files before so I knew exactly what to expect from it. It contains a file description followed by the actual genome. Storing that into a variable was easy. The FastQ file was something new to me. I created a read_FASTQ() function that takes in the file and stores all four lines into a list which was later append to another list. returning a list of lists contating the information wanted by me and needed by Dr. Asa Ben-Hur. The only information I needed was the ID and the pattern. Now that I have the contents needed from both files stored into 2 seprates variables, I am now ready to create my Read Map program.

# Data Processing
> Now that we got the easy part out of the way it's time to get the real work started. The first thing I do is call my map_reads function where all the data gets processed. Let me explain the image below. map_reads takes in five parameters. the first two being the data I collected from the two files. The third parameter is the name for the output file where I will write what my program ahs collected. The fourth parameter is the number of erros allowed when matching and lastly I pass it a boolean. If it's true then I want map_reads to create an output file, if False then I just want my code to add on the the CSV output file.


```
# MAIN
if __name__== '__main__':
  genome = read_fasta("sequence.fasta") # TEXT
  fastq = read_FASTQ("ecoli.fastq") # PATTERNS

  map_reads(genome, fastq, 'output.csv', 0, True)
  
  # Reverse Complement 
  r_genome = complement(genome)
  
  map_reads(r_genome, fastq, 'output.csv', 0, False)
```


>Now I want to talk about my function in two different parts. The very first thing I do is add a '$' character at the end of the genome text. This will help me identify the ending of the genome in the suffix array that will be created soon. Before that, I need to create the output file where my data is going to be stored unless if the boolean coming in is false, then I dont create a new file. After that I create two arrays. The first one being the suffix index. The second being the sorted suffix text. The suffix_array function was given to me by Dr. Asa Ben-Hur but the sorted suffix text function was created by me.


```
# Map Reads first half
def map_reads(text, patterns, output_file, num_errors=0, first=True) :
    # Add to the end of Genome
    text += '$'
    
    # create output file
    if first:
      df = pd.DataFrame(list())
      df.to_csv(output_file)
      with open(output_file, 'w', newline='') as file:
        col_names = ['Id', 'Pattern', 'Index(0 Based)']
        writer = csv.writer(file)
        writer.writerow(col_names)
        file.close()

    # Create suffix Array's
    sfxArray = suffix_array(text)
    print('finished suffix array')
    sfxSorted = sorted_suffix(text, sfxArray)
    print('finished suffix sorted')
```


```
# Creates Array of suffix Text
def sorted_suffix(text, sfxArray):
  return [text[i:] for i in sfxArray]
```
> The second part is where I go through each pattern in the patterns list and see if they belong in the genome text. I grab the first pattern and send it to a fucntion called binary_Search(). Binary search returns two things only. It will return -1 if the pattern was not found in the genome. It will return an int from 0 to len(genome). This intenger will help tell me where to find pattern in the genome index. If my program does find a pattern then it will go on and open the output file created and write into it 3 things. The Id of the pattern, the symbol '+' or'-', and the index of where the pattern appears in the genome.


```
# 2nd half
    for pattern in patterns:
      position = binary_Search(sfxSorted, pattern[1], num_errors)
      
      if(position != -1):
        with open(output_file, 'a', newline='') as file:
          writer = csv.writer(file)
          if first:
            temp = [pattern[0], '+', sfxArray[position]]
          else:
            temp = [pattern[0], '-', sfxArray[position]] 
            
          writer.writerow(temp)
          file.close()
```
>There are a lot of moving pieces in my code. So I do want to explain each one carefully. I want to start of by explaining my thought process for my binary search function. It takes in three parameters. The first one I pass it the suffix text sorted. The second parameter being the pattern that i'm trying to find. Lastly, its the number of errors the pattern can have. My binary search function is pretty much the same as any other one. We start at the middle and see if the element im looking for is at the top/left of my list or at the bottom/right of the list or of course are we right on the element I am looking for. The only diffrence is where I check for number of errors. It is located in the first part of the if statement. I first check if the number of errors is greater than 0. If it's not then im doing a basic binary search. But if it is greater then I also check if the number of mismatches between the pattern and text are equal to the number of errors. If it passes both of those statements then I move on to splitting up the text and pattern into num_errors + 1 pieces. 

```
# Searches through suffix Text to see if pattern is part of Text
def binary_Search(sfxSorted, pattern, num_errors=0):
  start = 0
  end = len(sfxSorted) - 1
  length = len(pattern)


  while start <= end:
    middle = (start + end) // 2
    test = sfxSorted[middle]
    midpoint = test[:length]

    # Check number of errors
    if (sum(c1!=c2 for c1,c2 in zip(pattern,midpoint)) == num_errors and num_errors > 0):
      split_pattern = divide_string(pattern, num_errors+1)
      split_text = divide_string(midpoint, num_errors+1)
      
      for i in range(len(split_text)):
        if split_pattern[i] == split_text[i]:
          return binary_Search(sfxSorted, split_pattern[i], 0)

    # Else if midpoint is greater, ignore top/left half
    elif(midpoint  > pattern):
      end = middle - 1
    
    # Else if x is smaller, ignore bottom/right half
    elif( midpoint < pattern):
      start = middle + 1
    
    # Checks if pattern is present at mid
    else:
      return top_search(sfxSorted, pattern, middle)

  return -1
```
>To divde the two strings up I created a function called divide_string(). I just take the string and divide into the correct number of pieces depending on the count of num_errors. Then it returns a list with the strings divided up.

```
# If mismatch found then it breaks string up into n pieces
def divide_string(string, n):
  chunk = int(len(string) / n) + (len(string) % n > 0)
  start = 0
  store = []
  for i in range(n):
    if i == n-1:
      store.append(string[start:])
    else:
      store.append(string[start:start+chunk])

    start += chunk

  return store
```
>Now I go through the 2 list that have the strings divided up and I check if list1[0] == list2[0]. If its true then those 2 match up and then I send that shorter pattern back into the binary search except this time I set num_errors to 0 because I know it has no erros so I just need to find it in the genome. When the pattern is found in the suffix text sorted, then I send the sorted list, pattern, and the location of it to a function called top_search(). All top_search() does it find the first occurence of the pattern inside the sorted suffix text. What I do first in it, is to shorten the the strings in the suffix sorted list to the size of the pattern because then it contains just the information that I need. I move up the index if the pattern is matching up with the suffix sorted text. One's it stops matching up then I know where the first occurence is and thats what I return back to binary search which then returns it back to map_reads() function.

```
# Top_search()
def top_search(sfxSorted, pattern, middle):
  shorten = [x[:len(pattern)] for x in sfxSorted]
  shorten = shorten[:middle+1]

  end = len(shorten)-1
  firstOccur = end

  while pattern == shorten[end]:
    firstOccur = end
    end -= 1
  
  return firstOccur
```

# Data Analysis
> Just to exlpain what data my program was able to collect for me, I am gonna run my code with only part of the reference genome text. Below are the parameters I used to collect just a sample.

```
  map_reads(genome[:3000], fastq, 'output.csv', 0, True)
  
  # Reverse Complement
  r_genome = complement(r_genome[:len(r_genome)-1])
  map_reads(r_genome[:3000], fastq, 'output.csv', 0, False)
```
>Whatever directory my program is in, that is where the output csv file with be created. Below we can see what was collect just with part of the genome.

```
Id,Pattern,Index(0 Based)
@EAS20_8_6_87_1257_1390/1,+,2699
@EAS20_8_6_87_1357_2002/1,+,2109
@EAS20_8_6_87_1255_1310/1,-,995
@EAS20_8_6_87_1261_275/1,-,1400
@EAS20_8_6_87_1316_23/1,-,669
```
>We can see that we got a collection of data. The headers describe what the data is. The first piece is the pattern ID, then we got the pattern Symbol. If it is '+' then that means it found a pattern with the genome untouched. If  it's '-' then that means a pattern was found with the reverse complement genome. Lastly we have the index of the pattern located in the genome. So the first row says that pattern Id: @EAS20_8_6_87_1257_1390/1 was found the genome in index 2699. So let me print out part of the genome at that index.


```
# Code plus output
print(genome[2699:2799])

output:
GCCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTGCCGGTGTCTTTGCTGATCTGCTACGTACCCTCTCATGGAAGTTAGGAGTCTGA
```
>Now lets go into the fastQ file and see if I can find that pattern ID and if it matches. I went into the fastQ file and did a quick ctr + f. I copied and pasted the pattern Id and this is what I found.

```
# FastQ file
@EAS20_8_6_87_1257_1390/1
GCCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTGCCGGTGTCTTTGCTGATCTGCTACGTACCCTCTCATGGAAGTTAGGAGTCTGA
+
HHGFHHFHHHHGHFG8FD=DHHHHHHEHGHHFGHH?EGFFABDFBEEC?H:1A=><DDB?D@4D:HHGHDGHHGG-FG?G;1<8D+2?############
```
>Now if we compare what pattern my program got and what the FastQ file has for that Id. We get this...


```
GCCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTGCCGGTGTCTTTGCTGATCTGCTACGTACCCTCTCATGGAAGTTAGGAGTCTGA
GCCGTTGGTACTGCGCGGATATGGTGCGGGCAATGACGTTACAGCTGCCGGTGTCTTTGCTGATCTGCTACGTACCCTCTCATGGAAGTTAGGAGTCTGA
```
>Now if we do the same thing with this row "@EAS20_8_6_87_1255_1310/1,-,995" we get.


```
# Code plus output
# Code plus output
print(r_genome[995:1095])

output:
ACGAAAGCTAGCATTTAGATACGATGATTTCATCAAACTGTTAACGTGCTACAATTGAACTTGATATATGTCAACGAAGCGTAGTTTTATTGGGTGTCCG
```
>Lets take a quick look at the FastQ file for this pattern.


```
# FastQ file
@EAS20_8_6_87_1255_1310/1
ACGAAAGCTAGCATTTAGATACGATGATTTCATCAAACTGTTAACGTGCTACAATTGAACTTGATATATGTCAACGAAGCGTAGTTTTATTGGGTGTCCG
+
FBFFEHHHHHHEHHHHHHHHHGGHHHHHHHFHHHGHHHHHGHHHFHHHGHHHGHGHHFHHHHFHHHHDHHHHHFFHGGHDHEHHHHGHHHCBHDEHHGEH
```
>Compare

```
ACGAAAGCTAGCATTTAGATACGATGATTTCATCAAACTGTTAACGTGCTACAATTGAACTTGATATATGTCAACGAAGCGTAGTTTTATTGGGTGTCCG
ACGAAAGCTAGCATTTAGATACGATGATTTCATCAAACTGTTAACGTGCTACAATTGAACTTGATATATGTCAACGAAGCGTAGTTTTATTGGGTGTCCG
```
>My program was able to find the location of the pattern in the reference genome using suffix array!





















### Submission

Submit your assignment as a Jupyter notebook via Canvas.  

### Grading 

Here is what the grade sheet will look like for this assignment. 

```
Grading sheet for assignment 1
- Correctness of map_reads (80 pts)
- Results analysis and discussion (20 pts)
```
